In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
import time

In [3]:
#df_drag
df_drag = pd.read_csv('/content/gdrive/MyDrive/날씨빅콘/busan_drag_add_final (1).csv')
df_ans = pd.read_csv('/content/gdrive/MyDrive/날씨빅콘/busan_answer2.csv', index_col = 0)

# drag데이터 날짜 시계열로 바꾸기

In [4]:
# 'busan_drag_train_final.time' 컬럼의 작은 따옴표('') 삭제
df_drag['busan_drag_train_final.time'] = df_drag['busan_drag_train_final.time'].str.replace('"', '')

In [5]:
df_drag.reset_index(drop=True, inplace=True)
df_drag

,busan_drag_train_final.num,busan_drag_train_final.time,busan_drag_train_final.latitude,busan_drag_train_final.longitude,busan_drag_train_final.sog,busan_drag_train_final.cog,busan_drag_train_final.hdg,stn_num
0,1,2021-01-14 22:47,"""N35.049747""","""E129.072122""",0.5,123.8,-99,N3
1,1,2021-01-14 22:50,"""N35.049485""","""E129.072438""",0.2,131.7,-99,N3
2,1,2021-01-14 22:56,"""N35.049452""","""E129.072440""",0.2,255.9,-99,N3
3,1,2021-01-14 23:02,"""N35.049402""","""E129.072257""",0.1,280.3,-99,N3
4,1,2021-01-14 23:11,"""N35.049100""","""E129.072215""",0.1,157.6,-99,N3
...,...,...,...,...,...,...,...,...
226481,87,2022-05-07 20:25,"""N35.053083""","""E129.061333""",1.3,26.0,97,N3
226482,87,2022-05-07 20:25,"""N35.053083""","""E129.061333""",1.3,29.0,99,N3
226483,87,2022-05-07 20:26,"""N35.053117""","""E129.061400""",1.2,48.0,106,N3
226484,87,2022-05-07 20:26,"""N35.053117""","""E129.061417""",1.2,52.0,108,N3


# ans데이터 시계열 컬럼인['busan_drag_train_final.time'] 만들고 ['target] 변수 생성

In [6]:
df_ans['busan_drag_train_final.time'] = df_ans['busan_answer.year'].astype(str) + '-' + df_ans['busan_answer.mon'].astype(str) + '-' + df_ans['busan_answer.day'].astype(str)+ ' ' + df_ans['busan_answer.hour'].astype(str)+ ':' + df_ans['busan_answer.min'].astype(str)

In [7]:
df_ans['busan_drag_train_final.time'] = pd.to_datetime(df_ans['busan_drag_train_final.time'])

In [8]:
df_ans['target'] = 1

In [9]:
df_ans.reset_index(drop=True, inplace=True)

In [10]:
# df_ans의 필요한 컬럼인 busan_drag_train_final.time, busan_answer.num, target 만 추출
df_ans = df_ans[['busan_drag_train_final.time', 'busan_answer.num', 'target']]

In [11]:
# df_drag와 조인하기 위해 df_ans 컬럼명을 df_drag명과 맞춰주기
df_ans = df_ans.rename(columns={'busan_answer.num': 'busan_drag_train_final.num'})

In [12]:
# 총 87개의 배의 닻끌림 인지 시각 확인
df_ans

,busan_drag_train_final.time,busan_drag_train_final.num,target
0,2021-01-17 02:04:00,1,1
1,2021-01-17 04:16:00,2,1
2,2021-01-17 04:56:00,3,1
3,2021-01-23 14:03:00,4,1
4,2021-01-23 15:18:00,5,1
...,...,...,...
82,2022-04-13 02:17:00,83,1
83,2022-04-22 13:53:00,84,1
84,2022-04-28 22:35:00,85,1
85,2022-05-07 19:53:00,86,1


In [13]:
from datetime import timedelta
ans_data = pd.DataFrame(columns=['busan_drag_train_final.time', 'busan_drag_train_final.num', 'target'])
uid = 0
for data1, data2 in zip(df_ans['busan_drag_train_final.time'].values, df_ans['busan_drag_train_final.num'].values):
  for i in range(1, 31):
    minus_data = pd.to_datetime(data1) - timedelta(minutes=i)
    plus_data = pd.to_datetime(data1) + timedelta(minutes=i)
    ans_data.loc[uid, 'busan_drag_train_final.time'] = minus_data
    ans_data.loc[uid+1, 'busan_drag_train_final.time'] = plus_data
    ans_data.loc[uid, 'busan_drag_train_final.num'] = data2
    ans_data.loc[uid+1, 'busan_drag_train_final.num'] = data2
    ans_data.loc[uid, 'target'] = 1
    ans_data.loc[uid+1, 'target'] = 1
    uid += 2

In [14]:
target_data=pd.concat([ans_data, df_ans])

In [15]:
target_data['busan_drag_train_final.num']=target_data['busan_drag_train_final.num'].astype(int)

In [16]:
target_data

,busan_drag_train_final.time,busan_drag_train_final.num,target
0,2021-01-17 02:03:00,1,1
1,2021-01-17 02:05:00,1,1
2,2021-01-17 02:02:00,1,1
3,2021-01-17 02:06:00,1,1
4,2021-01-17 02:01:00,1,1
...,...,...,...
82,2022-04-13 02:17:00,83,1
83,2022-04-22 13:53:00,84,1
84,2022-04-28 22:35:00,85,1
85,2022-05-07 19:53:00,86,1


In [17]:
df_drag

,busan_drag_train_final.num,busan_drag_train_final.time,busan_drag_train_final.latitude,busan_drag_train_final.longitude,busan_drag_train_final.sog,busan_drag_train_final.cog,busan_drag_train_final.hdg,stn_num
0,1,2021-01-14 22:47,"""N35.049747""","""E129.072122""",0.5,123.8,-99,N3
1,1,2021-01-14 22:50,"""N35.049485""","""E129.072438""",0.2,131.7,-99,N3
2,1,2021-01-14 22:56,"""N35.049452""","""E129.072440""",0.2,255.9,-99,N3
3,1,2021-01-14 23:02,"""N35.049402""","""E129.072257""",0.1,280.3,-99,N3
4,1,2021-01-14 23:11,"""N35.049100""","""E129.072215""",0.1,157.6,-99,N3
...,...,...,...,...,...,...,...,...
226481,87,2022-05-07 20:25,"""N35.053083""","""E129.061333""",1.3,26.0,97,N3
226482,87,2022-05-07 20:25,"""N35.053083""","""E129.061333""",1.3,29.0,99,N3
226483,87,2022-05-07 20:26,"""N35.053117""","""E129.061400""",1.2,48.0,106,N3
226484,87,2022-05-07 20:26,"""N35.053117""","""E129.061417""",1.2,52.0,108,N3


In [18]:
df_drag['timestamp'] = pd.to_datetime(df_drag['busan_drag_train_final.time']).dt.strftime('%Y-%m-%d %H:%M')
target_data['timestamp'] = pd.to_datetime(target_data['busan_drag_train_final.time']).dt.strftime('%Y-%m-%d %H:%M')

In [19]:
merge_data=pd.merge(df_drag, target_data, on=['busan_drag_train_final.num', 'timestamp'], how='left')

In [20]:
merge_data=merge_data.fillna(0)

In [21]:
merge_data

,busan_drag_train_final.num,busan_drag_train_final.time_x,busan_drag_train_final.latitude,busan_drag_train_final.longitude,busan_drag_train_final.sog,busan_drag_train_final.cog,busan_drag_train_final.hdg,stn_num,timestamp,busan_drag_train_final.time_y,target
0,1,2021-01-14 22:47,"""N35.049747""","""E129.072122""",0.5,123.8,-99,N3,2021-01-14 22:47,0,0
1,1,2021-01-14 22:50,"""N35.049485""","""E129.072438""",0.2,131.7,-99,N3,2021-01-14 22:50,0,0
2,1,2021-01-14 22:56,"""N35.049452""","""E129.072440""",0.2,255.9,-99,N3,2021-01-14 22:56,0,0
3,1,2021-01-14 23:02,"""N35.049402""","""E129.072257""",0.1,280.3,-99,N3,2021-01-14 23:02,0,0
4,1,2021-01-14 23:11,"""N35.049100""","""E129.072215""",0.1,157.6,-99,N3,2021-01-14 23:11,0,0
...,...,...,...,...,...,...,...,...,...,...,...
226481,87,2022-05-07 20:25,"""N35.053083""","""E129.061333""",1.3,26.0,97,N3,2022-05-07 20:25,2022-05-07 20:25:00,1
226482,87,2022-05-07 20:25,"""N35.053083""","""E129.061333""",1.3,29.0,99,N3,2022-05-07 20:25,2022-05-07 20:25:00,1
226483,87,2022-05-07 20:26,"""N35.053117""","""E129.061400""",1.2,48.0,106,N3,2022-05-07 20:26,2022-05-07 20:26:00,1
226484,87,2022-05-07 20:26,"""N35.053117""","""E129.061417""",1.2,52.0,108,N3,2022-05-07 20:26,2022-05-07 20:26:00,1


In [23]:
# hdg 결측치(-99 등등 결측치로 변경)
merge_data['busan_drag_train_final.hdg'].value_counts()

-99     15912
 68      1655
 69      1628
 64      1602
 66      1600
        ...  
 156      269
 292      268
 310      267
 321      266
 297      263
Name: busan_drag_train_final.hdg, Length: 361, dtype: int64

In [25]:
merge_data.replace(-99, np.nan, inplace=True)
merge_data.replace(-99.9, np.nan, inplace=True)
merge_data.replace(-999, np.nan, inplace=True)

In [26]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226486 entries, 0 to 226485
Data columns (total 11 columns):
 #   Column                            Non-Null Count   Dtype  
---  ------                            --------------   -----  
 0   busan_drag_train_final.num        226486 non-null  int64  
 1   busan_drag_train_final.time_x     226486 non-null  object 
 2   busan_drag_train_final.latitude   226486 non-null  object 
 3   busan_drag_train_final.longitude  226486 non-null  object 
 4   busan_drag_train_final.sog        226486 non-null  float64
 5   busan_drag_train_final.cog        222180 non-null  float64
 6   busan_drag_train_final.hdg        210574 non-null  float64
 7   stn_num                           226486 non-null  object 
 8   timestamp                         226486 non-null  object 
 9   busan_drag_train_final.time_y     226486 non-null  object 
 10  target                            226486 non-null  int64  
dtypes: float64(3), int64(2), object(6)
memory usage: 20.

In [27]:
#df_anch
df_anch = pd.read_csv('/content/gdrive/MyDrive/날씨빅콘/busan_anch_add_final.csv')
df_anch

,busan_anch_train_final.num,busan_anch_train_final.time,busan_anch_train_final.latitude,busan_anch_train_final.longitude,busan_anch_train_final.sog,busan_anch_train_final.cog,busan_anch_train_final.hdg,stn_num
0,1,"""2021-01-08 0:39""","""N35.063823""","""E129.053027""",0.1,-99.9,"""344""",N3
1,1,"""2021-01-08 0:39""","""N35.063828""","""E129.053030""",0.2,-99.9,"""342""",N3
2,1,"""2021-01-08 0:39""","""N35.063833""","""E129.053032""",0.2,-99.9,"""339""",N3
3,1,"""2021-01-08 0:39""","""N35.063847""","""E129.053033""",0.1,-99.9,"""337""",N3
4,1,"""2021-01-08 0:39""","""N35.063862""","""E129.053032""",0.2,-99.9,"""336""",N3
...,...,...,...,...,...,...,...,...
67742,160,"""2022-04-10 17:36:26""","""N35.048752""","""E129.042125""",0.1,125.3,"""222""",N3
67743,160,"""2022-04-10 17:39:26""","""N35.048738""","""E129.042160""",0.1,149.6,"""223""",N3
67744,160,"""2022-04-10 17:45:26""","""N35.048702""","""E129.042170""",0.1,152.4,"""223""",N3
67745,160,"""2022-04-10 17:48:26""","""N35.048752""","""E129.042160""",0.1,172.6,"""223""",N3


In [28]:
# 'busan_anch_train_final.time' 컬럼의 작은 따옴표('') 삭제
df_anch['busan_anch_train_final.time'] = df_anch['busan_anch_train_final.time'].str.replace('"', '')

In [29]:
df_anch['busan_anch_train_final.time'] = pd.to_datetime(df_anch['busan_anch_train_final.time'])

In [30]:
# df_anch에 없는 timestamp추가
df_anch['timestamp'] = pd.to_datetime(df_anch['busan_anch_train_final.time']).dt.strftime('%Y-%m-%d %H:%M')

In [31]:
df_anch['target'] = 0

In [32]:
df_anch

,busan_anch_train_final.num,busan_anch_train_final.time,busan_anch_train_final.latitude,busan_anch_train_final.longitude,busan_anch_train_final.sog,busan_anch_train_final.cog,busan_anch_train_final.hdg,stn_num,timestamp,target
0,1,2021-01-08 00:39:00,"""N35.063823""","""E129.053027""",0.1,-99.9,"""344""",N3,2021-01-08 00:39,0
1,1,2021-01-08 00:39:00,"""N35.063828""","""E129.053030""",0.2,-99.9,"""342""",N3,2021-01-08 00:39,0
2,1,2021-01-08 00:39:00,"""N35.063833""","""E129.053032""",0.2,-99.9,"""339""",N3,2021-01-08 00:39,0
3,1,2021-01-08 00:39:00,"""N35.063847""","""E129.053033""",0.1,-99.9,"""337""",N3,2021-01-08 00:39,0
4,1,2021-01-08 00:39:00,"""N35.063862""","""E129.053032""",0.2,-99.9,"""336""",N3,2021-01-08 00:39,0
...,...,...,...,...,...,...,...,...,...,...
67742,160,2022-04-10 17:36:26,"""N35.048752""","""E129.042125""",0.1,125.3,"""222""",N3,2022-04-10 17:36,0
67743,160,2022-04-10 17:39:26,"""N35.048738""","""E129.042160""",0.1,149.6,"""223""",N3,2022-04-10 17:39,0
67744,160,2022-04-10 17:45:26,"""N35.048702""","""E129.042170""",0.1,152.4,"""223""",N3,2022-04-10 17:45,0
67745,160,2022-04-10 17:48:26,"""N35.048752""","""E129.042160""",0.1,172.6,"""223""",N3,2022-04-10 17:48,0


In [33]:
df_anch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67747 entries, 0 to 67746
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   busan_anch_train_final.num        67747 non-null  int64         
 1   busan_anch_train_final.time       67747 non-null  datetime64[ns]
 2   busan_anch_train_final.latitude   67747 non-null  object        
 3   busan_anch_train_final.longitude  67747 non-null  object        
 4   busan_anch_train_final.sog        67747 non-null  float64       
 5   busan_anch_train_final.cog        67747 non-null  float64       
 6   busan_anch_train_final.hdg        67747 non-null  object        
 7   stn_num                           67747 non-null  object        
 8   timestamp                         67747 non-null  object        
 9   target                            67747 non-null  int64         
dtypes: datetime64[ns](1), float64(2), int64(2), ob

In [36]:
# hdg 결측치(-99 등등 결측치로 변경)
df_anch['busan_anch_train_final.hdg'] = df_anch['busan_anch_train_final.hdg'].str.replace('"', '')

In [39]:
df_anch['busan_anch_train_final.hdg'].replace('-99.-', '-99', inplace=True)

In [41]:
# 수치형 자료로 변환
df_anch['busan_anch_train_final.hdg'] = df_anch['busan_anch_train_final.hdg'].astype(float)

In [43]:
df_anch['busan_anch_train_final.hdg'].replace(-99, np.nan, inplace=True)
df_anch['busan_anch_train_final.hdg'].replace(-99.9, np.nan, inplace=True)
df_anch['busan_anch_train_final.hdg'].replace(-99.0, np.nan, inplace=True)
df_anch['busan_anch_train_final.hdg'].replace(-999, np.nan, inplace=True)

In [44]:
df_anch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67747 entries, 0 to 67746
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   busan_anch_train_final.num        67747 non-null  int64         
 1   busan_anch_train_final.time       67747 non-null  datetime64[ns]
 2   busan_anch_train_final.latitude   67747 non-null  object        
 3   busan_anch_train_final.longitude  67747 non-null  object        
 4   busan_anch_train_final.sog        67747 non-null  float64       
 5   busan_anch_train_final.cog        67747 non-null  float64       
 6   busan_anch_train_final.hdg        67481 non-null  float64       
 7   stn_num                           67747 non-null  object        
 8   timestamp                         67747 non-null  object        
 9   target                            67747 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2), ob

In [45]:
merge_data.drop(['busan_drag_train_final.time_y'], axis=1, inplace=True)
merge_data.rename({'busan_drag_train_final.time_x':'busan_drag_train_final.time'}, axis=1, inplace=1)

In [46]:
merge_data['busan_drag_train_final.time'] = pd.to_datetime(merge_data['busan_drag_train_final.time'])

In [47]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226486 entries, 0 to 226485
Data columns (total 10 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   busan_drag_train_final.num        226486 non-null  int64         
 1   busan_drag_train_final.time       226486 non-null  datetime64[ns]
 2   busan_drag_train_final.latitude   226486 non-null  object        
 3   busan_drag_train_final.longitude  226486 non-null  object        
 4   busan_drag_train_final.sog        226486 non-null  float64       
 5   busan_drag_train_final.cog        222180 non-null  float64       
 6   busan_drag_train_final.hdg        210574 non-null  float64       
 7   stn_num                           226486 non-null  object        
 8   timestamp                         226486 non-null  object        
 9   target                            226486 non-null  int64         
dtypes: datetime64[ns](1), float64(3)

In [48]:
# anch와 drag 시계열 컬럼 이름 맞추기
df_anch = df_anch.rename(columns={'busan_anch_train_final.num': 'busan_drag_train_final.num','busan_anch_train_final.time':'busan_drag_train_final.time','busan_anch_train_final.latitude':'busan_drag_train_final.latitude','busan_anch_train_final.longitude':'busan_drag_train_final.longitude','busan_anch_train_final.sog':'busan_drag_train_final.sog','busan_anch_train_final.cog':'busan_drag_train_final.cog','busan_anch_train_final.hdg':'busan_drag_train_final.hdg'})

In [49]:
df_anch.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 67747 entries, 0 to 67746
Data columns (total 10 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   busan_drag_train_final.num        67747 non-null  int64         
 1   busan_drag_train_final.time       67747 non-null  datetime64[ns]
 2   busan_drag_train_final.latitude   67747 non-null  object        
 3   busan_drag_train_final.longitude  67747 non-null  object        
 4   busan_drag_train_final.sog        67747 non-null  float64       
 5   busan_drag_train_final.cog        67747 non-null  float64       
 6   busan_drag_train_final.hdg        67481 non-null  float64       
 7   stn_num                           67747 non-null  object        
 8   timestamp                         67747 non-null  object        
 9   target                            67747 non-null  int64         
dtypes: datetime64[ns](1), float64(3), int64(2), ob

In [50]:
merge_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 226486 entries, 0 to 226485
Data columns (total 10 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   busan_drag_train_final.num        226486 non-null  int64         
 1   busan_drag_train_final.time       226486 non-null  datetime64[ns]
 2   busan_drag_train_final.latitude   226486 non-null  object        
 3   busan_drag_train_final.longitude  226486 non-null  object        
 4   busan_drag_train_final.sog        226486 non-null  float64       
 5   busan_drag_train_final.cog        222180 non-null  float64       
 6   busan_drag_train_final.hdg        210574 non-null  float64       
 7   stn_num                           226486 non-null  object        
 8   timestamp                         226486 non-null  object        
 9   target                            226486 non-null  int64         
dtypes: datetime64[ns](1), float64(3)

In [51]:
merged_df = pd.concat([merge_data, df_anch], axis=0)

In [52]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 294233 entries, 0 to 67746
Data columns (total 10 columns):
 #   Column                            Non-Null Count   Dtype         
---  ------                            --------------   -----         
 0   busan_drag_train_final.num        294233 non-null  int64         
 1   busan_drag_train_final.time       294233 non-null  datetime64[ns]
 2   busan_drag_train_final.latitude   294233 non-null  object        
 3   busan_drag_train_final.longitude  294233 non-null  object        
 4   busan_drag_train_final.sog        294233 non-null  float64       
 5   busan_drag_train_final.cog        289927 non-null  float64       
 6   busan_drag_train_final.hdg        278055 non-null  float64       
 7   stn_num                           294233 non-null  object        
 8   timestamp                         294233 non-null  object        
 9   target                            294233 non-null  int64         
dtypes: datetime64[ns](1), float64(3),

In [54]:
merged_df

,busan_drag_train_final.num,busan_drag_train_final.time,busan_drag_train_final.latitude,busan_drag_train_final.longitude,busan_drag_train_final.sog,busan_drag_train_final.cog,busan_drag_train_final.hdg,stn_num,timestamp,target
0,1,2021-01-14 22:47:00,"""N35.049747""","""E129.072122""",0.5,123.8,NaN,N3,2021-01-14 22:47,0
1,1,2021-01-14 22:50:00,"""N35.049485""","""E129.072438""",0.2,131.7,NaN,N3,2021-01-14 22:50,0
2,1,2021-01-14 22:56:00,"""N35.049452""","""E129.072440""",0.2,255.9,NaN,N3,2021-01-14 22:56,0
3,1,2021-01-14 23:02:00,"""N35.049402""","""E129.072257""",0.1,280.3,NaN,N3,2021-01-14 23:02,0
4,1,2021-01-14 23:11:00,"""N35.049100""","""E129.072215""",0.1,157.6,NaN,N3,2021-01-14 23:11,0
...,...,...,...,...,...,...,...,...,...,...
67742,160,2022-04-10 17:36:26,"""N35.048752""","""E129.042125""",0.1,125.3,222.0,N3,2022-04-10 17:36,0
67743,160,2022-04-10 17:39:26,"""N35.048738""","""E129.042160""",0.1,149.6,223.0,N3,2022-04-10 17:39,0
67744,160,2022-04-10 17:45:26,"""N35.048702""","""E129.042170""",0.1,152.4,223.0,N3,2022-04-10 17:45,0
67745,160,2022-04-10 17:48:26,"""N35.048752""","""E129.042160""",0.1,172.6,223.0,N3,2022-04-10 17:48,0


In [55]:
merged_df.reset_index(drop=True, inplace=True)

In [56]:
merged_df

,busan_drag_train_final.num,busan_drag_train_final.time,busan_drag_train_final.latitude,busan_drag_train_final.longitude,busan_drag_train_final.sog,busan_drag_train_final.cog,busan_drag_train_final.hdg,stn_num,timestamp,target
0,1,2021-01-14 22:47:00,"""N35.049747""","""E129.072122""",0.5,123.8,NaN,N3,2021-01-14 22:47,0
1,1,2021-01-14 22:50:00,"""N35.049485""","""E129.072438""",0.2,131.7,NaN,N3,2021-01-14 22:50,0
2,1,2021-01-14 22:56:00,"""N35.049452""","""E129.072440""",0.2,255.9,NaN,N3,2021-01-14 22:56,0
3,1,2021-01-14 23:02:00,"""N35.049402""","""E129.072257""",0.1,280.3,NaN,N3,2021-01-14 23:02,0
4,1,2021-01-14 23:11:00,"""N35.049100""","""E129.072215""",0.1,157.6,NaN,N3,2021-01-14 23:11,0
...,...,...,...,...,...,...,...,...,...,...
294228,160,2022-04-10 17:36:26,"""N35.048752""","""E129.042125""",0.1,125.3,222.0,N3,2022-04-10 17:36,0
294229,160,2022-04-10 17:39:26,"""N35.048738""","""E129.042160""",0.1,149.6,223.0,N3,2022-04-10 17:39,0
294230,160,2022-04-10 17:45:26,"""N35.048702""","""E129.042170""",0.1,152.4,223.0,N3,2022-04-10 17:45,0
294231,160,2022-04-10 17:48:26,"""N35.048752""","""E129.042160""",0.1,172.6,223.0,N3,2022-04-10 17:48,0


In [58]:
#merged_df.to_csv('/content/gdrive/MyDrive/날씨빅콘/busan_all_stn_taget.csv')